In [ ]:
from google.colab import drive
drive.mount('/content/drive')
katago = pd.read_csv('/content/drive/MyDrive/data_katago.csv')
katago.dropna(how = 'any', inplace= True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(katago.columns)

Index(['W_rating', 'B_rating', 'W_nickname', 'B_nickname', 'Result', 'W_move',
       'B_move', 'W_scoreLead', 'B_scoreLead', 'W_scoreSelfplay',
       'B_scoreSelfplay', 'W_scoreStdev', 'B_scoreStdev', 'W_utility',
       'B_utility', 'W_visits', 'B_visits', 'W_winrate', 'B_winrate'],
      dtype='object')


In [ ]:
class MovesInfo:
    def __init__(self, row, n_moves=None, player='W'):
        enemy = 'B' if player == 'W' else 'W'
        moves_len = min(len(row[player + '_winrate']), len(row[enemy + '_winrate']))
        if n_moves is None:
            start_ind = 0
        else:
            start_ind = max(moves_len - n_moves - 1, 0)

        end_ind = moves_len
        w_0 = []
        s_0 = []
        std_0 = []
        sf_0 = []
        ut_0 = []
        if player == 'B':
          w_0 = [0.5]
          s_0 = [0.0]
          std_0 = [20.0]
          sf_0 = [0.0]
          ut_0 = [0.0]
        self.winrate_delta = np.array(row[player + '_winrate'][start_ind:end_ind]) - np.array(
            (w_0 + row[enemy + '_winrate'])[start_ind:end_ind])
        self.score_delta = np.array(row[player + '_scoreLead'][start_ind:end_ind]) - np.array(
            (s_0 + row[enemy + '_scoreLead'])[start_ind:end_ind])
        self.utility_delta = np.array(row[player + '_utility'][start_ind:end_ind]) - np.array(
            (ut_0 + row[enemy + '_utility'])[start_ind:end_ind])
        self.selfplay_delta = np.array(row[player + '_scoreSelfplay'][start_ind:end_ind]) - np.array(
            (sf_0 + row[enemy + '_scoreSelfplay'])[start_ind:end_ind])
        self.stddev_delta = np.array(row[player + '_scoreStdev'][start_ind:end_ind]) - np.array(
            (std_0 + row[enemy + '_scoreStdev'])[start_ind:end_ind])
        if player == 'B':
          self.winrate_delta = -self.winrate_delta
          self.score_delta = -self.score_delta
          self.selfplay_delta = -self.selfplay_delta
        self.move = row[player + '_move'].split()
        self.cnt_moves = end_ind - start_ind

def add_meta(row):
    if row['Result'] == '?':
        row['game_result'] = 0
    else:
        row['game_result'] = int(row['Result'])
    row['rank'] = get_int_from_rank(row['W_rating'])
    row['game_length'] = len(row['W_move']) + len(row['B_move'])

def get_unique_falls(df):
   convert_to_lists(df)
   df['mean_score'] = None 
   df['mean_unique_score'] = None
   df['rank'] = None
   df['game_length'] = None
   df['game_result'] = None
   df['mean_fall_W'] = None
   df['mean_fall_B'] = None
   for i, row in tqdm(df.iterrows()):
        row['rank'] = 1
        add_meta(row)
        W_moves = MovesInfo(row, player = 'W')
        B_moves = MovesInfo(row, player = 'B')
        row['mean_score'] = np.mean(W_moves.score_delta)
        row['mean_fall_W'] = np.mean(W_moves.score_delta)
        row['mean_fall_B'] = np.mean(B_moves.score_delta)
        falls = [0]
        for j in range(1, W_moves.cnt_moves - 1):
          #print(B_moves.score_delta[j])
          if W_moves.score_delta[j] < 0:
            prev = W_moves.score_delta[j - 1]
            cur = W_moves.score_delta[j]
            if B_moves.score_delta[j] < 0 and abs(B_moves.score_delta[j]) < 0.5 * abs(cur) and abs(cur):
              falls.append(W_moves.score_delta[j])
              print(W_moves.score_delta[j - 1], W_moves.score_delta[j])
              #print(row['W_scoreLead'][j - 1], row['B_scoreLead'][j], row['W_scoreLead'][j], row['B_scoreLead'][j] - row['W_scoreLead'][j - 1], W_moves.score_delta[j])
        row['mean_unique_score'] = np.mean(falls)    
   return df       

ktg =  get_unique_falls(katago[:4].copy())
from scipy.stats import pearsonr    
print(pearsonr(ktg['rank'], ktg['mean_score']))
print(pearsonr(ktg['rank'], ktg['mean_unique_score']))   

4it [00:00, 985.85it/s]
4it [00:00, 645.25it/s]

-0.6933093969999999 -2.451866238
-2.451866238 -1.15105534
-1.15105534 -2.9157044799999996
-1.15895355 -1.5970804700000003
-4.78564012 -3.4880155946
-0.19937893699999998 -0.9526096730000001
-0.537156527 -1.18761861
-0.24748396999999933 -1.2011232400000003
-0.015822879999999984 -1.5073513999999992
-0.08786249999999995 -1.3608636799999996
-0.4518699399999999 -1.2173371500000005
-1.1581373 -3.490897200000001
-0.0902472000000003 -1.5922661000000016
-0.9179458999999994 -2.1780643500000014
-0.2499922546 -0.41888444099999994
-0.1338990181 -0.9372389913
-0.8742676767 -1.259544931
-0.596470572 -0.925956057
-0.36464845999999973 -4.599550299999999
-8.5390082 -0.9362564000000013
-0.9362564000000013 -0.7319116000000001
2.1899337999999986 -14.691932699999999
-2.111405399999999 -0.9430103000000045
0.15034670000000006 -2.491750800000002
-5.178472599999999 -7.179523400000001
-0.9856015000000014 -3.7174643999999972
-0.6340313000000037 -2.2923469000000054
1.3103466000000026 -2.0460497999999987
-3.16493419

In [ ]:
print(np.mean(ktg['mean_fall_W']))
print(np.mean(ktg['mean_fall_B']))

-1.9622440319583065
-1.9347484381655145


In [ ]:
katago_new = get_df_with_lists(katago)

144204it [01:30, 1591.56it/s]
144204it [02:51, 842.06it/s]
144204it [01:02, 2319.71it/s]
144204it [01:49, 1315.17it/s]
144204it [01:56, 1234.53it/s]
